In [1]:
import pandas as pd 
import numpy as np 
import scipy as sp 

import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession

# Importar funciones necesarias
from pyspark.sql.functions import col, regexp_replace, dayofmonth, weekofyear, year, month
from pyspark.sql.functions import col, to_date, sum
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql.functions import regexp_replace, col, when
from pyspark.sql.functions import format_number
from pyspark.sql.types import IntegerType,FloatType
# Puedes obtener estadísticas específicas para una columna
from pyspark.sql.functions import rand, mean, min, max, lit
from pyspark.sql.functions import approx_count_distinct
from pyspark.sql.window import Window
from pyspark.sql.functions import log1p
from pyspark.sql import functions as F

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [2]:

# Inicializar la sesión de Spark como una variable global
global spark
spark = SparkSession.builder.appName("Appetl").getOrCreate()

# Definir variables globales para las rutas de archivos
global accepted_r, rejected_r
accepted_r = 'hdfs://namenode:8020/datasets/raw/accepted_2007_to_2018Q4.csv.gz'
rejected_r = 'hdfs://namenode:8020/datasets/raw/rejected_2007_to_2018Q4.csv.gz'

# Función para cargar los datos
def load_data():
    global accepted_df, rejected_df, accepted_dfm, rejected_dfm
    # Leer los archivos CSV comprimidos usando PySpark
    accepted_df = spark.read.option("header", "true").option("inferSchema", "true").csv(accepted_r)
    rejected_df = spark.read.option("header", "true").option("inferSchema", "true").csv(rejected_r)

    # Haciendo una copia del DataFrame usando alias
    accepted_dfm = accepted_df.alias("accepted_dfm")
    rejected_dfm = rejected_df.alias("rejected_dfm")

# Llamar a la función para cargar los datos
load_data()

# Puedes acceder a accepted_df, rejected_df, accepted_dfm, rejected_dfm desde cualquier parte del notebook.


In [3]:
# Función para limpiar y transformar los datos
def clean_and_transform():
    global rejected_dfm
    # Convertir columnas a tipos de datos adecuados y realizar transformaciones 
    rejected_dfm = rejected_dfm.withColumn("Risk_Score", col("Risk_Score").cast("float"))
    rejected_dfm = rejected_dfm.withColumn("Debt-To-Income Ratio", regexp_replace(col("Debt-To-Income Ratio"), "%", "").cast("float") / 100.0)
    rejected_dfm = rejected_dfm.withColumn("Zip Code", regexp_replace(col("Zip Code"), "[^0-9]", ""))
    rejected_dfm = rejected_dfm.withColumn("Policy Code", col("Policy Code").cast("int"))
    # Convertir "Amount Requested" a tipo fecha
    #rejected_dfm = rejected_dfm.withColumn("Application Date", to_date(col("Application Date"), "MM/dd/yyyy"))
    
    # Crear nuevas columnas para día, semana y mes
    rejected_dfm = rejected_dfm.withColumn('Day', dayofmonth(col('Application Date')))
    rejected_dfm = rejected_dfm.withColumn('Week', weekofyear(col('Application Date')))
    rejected_dfm = rejected_dfm.withColumn('Month', month(col('Application Date')))

In [4]:
clean_and_transform()

In [5]:
#rejected_dfm.printSchema()

In [6]:
#rejected_dfm.show()

In [7]:
#rejected_dfm.printSchema()

In [8]:
# Variables globales
mean_risk_score = None
mode_amount_requested = None

def handle_numeric_variables(df):
    global mean_risk_score
    global mode_amount_requested

    # Calcular la media de Risk_Score
    mean_risk_score = df.select(mean("Risk_Score")).collect()[0][0]

    # Calcular la moda de Amount Requested
    mode_amount_requested = df.groupBy("Amount Requested").count().orderBy(col("count").desc()).select("Amount Requested").first()[0]

    # Reemplazar nulos en Risk_Score por la media
    df = df.withColumn("Risk_Score", when(col("Risk_Score").isNull(), lit(mean_risk_score)).otherwise(col("Risk_Score")))

    # Reemplazar nulos en Amount Requested por la moda
    df = df.withColumn("Amount Requested", when(col("Amount Requested").isNull(), lit(mode_amount_requested)).otherwise(col("Amount Requested")))

    return df

In [9]:
def handle_categorical_variables(df):
    # Reemplazar los nulos en Loan Title con "Sin registro"
    df = df.withColumn("Loan Title", when(col("Loan Title").isNull(), "Sin registro").otherwise(col("Loan Title")))

    # Reemplazar los nulos en Zip Code con 999
    df = df.withColumn("Zip Code", when(col("Zip Code").isNull(), "999").otherwise(col("Zip Code")))

    # Reemplazar los nulos en State con "Sin registro"
    df = df.withColumn("State", when(col("State").isNull(), "Sin registro").otherwise(col("State")))

    # Reemplazar los nulos en Employment Length con "Sin registro"
    df = df.withColumn("Employment Length", when(col("Employment Length").isNull(), "Sin registro").otherwise(col("Employment Length")))

    return df

In [10]:
# Tratar variables numéricas
rejected_dfm = handle_numeric_variables(rejected_dfm)

# Tratar variables categóricas
rejected_dfm = handle_categorical_variables(rejected_dfm)


In [11]:
def split_train_test(df, train_percentage=0.8):
    """
    Divide un DataFrame en conjuntos de entrenamiento y prueba.

    Parámetros:
    - df: DataFrame de PySpark.
    - train_percentage: Porcentaje del DataFrame para el conjunto de entrenamiento.

    Retorna:
    - (train_df, test_df): Tupla de DataFrames para entrenamiento y prueba.
    """

    # Agrega una columna 'split' con valores aleatorios entre 0 y 1
    df_with_split = df.withColumn('split', rand())

    # Divide el DataFrame en train y test según el valor en la columna 'split'
    train_df = df_with_split.filter(col('split') <= train_percentage)
    test_df = df_with_split.filter(col('split') > train_percentage)

    # Elimina la columna 'split' ya que ya no es necesaria
    train_df = train_df.drop('split')
    test_df = test_df.drop('split')

    return train_df, test_df

# Llama a la función con tu DataFrame y porcentaje deseado
train_df, test_df = split_train_test(rejected_dfm, train_percentage=0.8)


In [25]:
# Definir la ubicación del almacén Hive en HDFS
warehouse_location = "/user/hive/warehouse"

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Test Hive Connection") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

# Ejecutar una consulta simple
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  lending|
+---------+



In [27]:
# Define la base de datos y la tabla
lending_database = "lending"

lending_train = "train_df"  # Reemplaza con el nombre real de tu tabla
lending_test = "test_df"  # Reemplaza con el nombre real de tu tabla

# Cambiar a la base de datos 'lending'
spark.sql(f"CREATE DATABASE IF NOT EXISTS {lending_database}")
spark.sql(f"USE {lending_database}")

# Guardar el DataFrame en Hive
train_df.write.mode("overwrite").saveAsTable(lending_train)
test_df.write.mode("overwrite").saveAsTable(lending_test)

In [28]:
# Ejecutar una consulta simple
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|  lending|
+---------+



In [29]:
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  lending|  test_df|      false|
|  lending| train_df|      false|
+---------+---------+-----------+



In [30]:
# Reemplaza 'tu_tabla' con el nombre de la tabla que deseas explorar
tabla_seleccionada = 'test_df'
spark.sql(f"DESCRIBE EXTENDED {tabla_seleccionada}").show(truncate=False)


+----------------------------+----------------------------+-------+
|col_name                    |data_type                   |comment|
+----------------------------+----------------------------+-------+
|Amount Requested            |double                      |NULL   |
|Application Date            |date                        |NULL   |
|Loan Title                  |string                      |NULL   |
|Risk_Score                  |double                      |NULL   |
|Debt-To-Income Ratio        |double                      |NULL   |
|Zip Code                    |string                      |NULL   |
|State                       |string                      |NULL   |
|Employment Length           |string                      |NULL   |
|Policy Code                 |int                         |NULL   |
|Day                         |int                         |NULL   |
|Week                        |int                         |NULL   |
|Month                       |int               

In [31]:
# Reemplaza 'tu_tabla' con el nombre de la tabla que deseas explorar
tabla_seleccionada = 'lending.train_df'
spark.sql(f"DESCRIBE EXTENDED {tabla_seleccionada}").show(truncate=False)

+----------------------------+----------------------------+-------+
|col_name                    |data_type                   |comment|
+----------------------------+----------------------------+-------+
|Amount Requested            |double                      |NULL   |
|Application Date            |date                        |NULL   |
|Loan Title                  |string                      |NULL   |
|Risk_Score                  |double                      |NULL   |
|Debt-To-Income Ratio        |double                      |NULL   |
|Zip Code                    |string                      |NULL   |
|State                       |string                      |NULL   |
|Employment Length           |string                      |NULL   |
|Policy Code                 |int                         |NULL   |
|Day                         |int                         |NULL   |
|Week                        |int                         |NULL   |
|Month                       |int               